In [5]:
!pip install category_encoders

In [6]:
import pandas as pd
import numpy as np
import os
import datetime
import tensorflow as tf
import category_encoders as ce
from sklearn.preprocessing import OneHotEncoder
import math
import pickle

from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Flatten, Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.optimizers import Adam

In [7]:
def wrangle(X):

  X = X.filter(['log_price', 'room_type', 'accommodates', 'bathrooms', 'latitude',
                'longitude','review_scores_rating', 'bedrooms', 'beds', 'TV', 
                'zipcode'], axis=1)

  X["zipcode"] = X["zipcode"].replace("Near 91304", 91304)
  X["zipcode"] = X["zipcode"].replace("1m", 10023)
  X["zipcode"] = pd.to_numeric(X["zipcode"], errors="coerce")
  X["zipcode"] = X["zipcode"].replace(np.nan, X["zipcode"].median())
  X["zipcode"] = X["zipcode"].astype(str)
  X['zipcode'] = X['zipcode'][:5] 
  X["zipcode"] = X["zipcode"].replace(".", "")  
  X["zipcode"] = X["zipcode"].astype(float)

  X = X.fillna(0)

  encoder = ce.OrdinalEncoder()
  X = encoder.fit_transform(X)

  X = X.astype(float)

  return X

In [8]:
df = pd.read_csv('clean.csv')

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (8,19,20,21,22,23,24,25,26,27) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [9]:
df = wrangle(df)

In [10]:
train, test = train_test_split(df, test_size=0.2, random_state=42)


In [11]:
X_train = train.drop(columns="log_price")
y_train = train["log_price"]

X_test = test.drop(columns="log_price")
y_test = test["log_price"]

In [21]:
adam = Adam(learning_rate=0.01)

model = Sequential([
    Dense(512, input_dim=10, activation= 'relu'),
    Dense(256, activation="relu"),
    Dense(128, activation="relu"),
    Dense(64, activation='relu'),
    Dense(32, activation='relu'),
    Dense(16, activation='relu'),
    Dense(1)
    ])

model.compile(loss='mean_squared_error', optimizer=adam, metrics=['mse'])

In [22]:
model.fit(x=X_train, 
          y=y_train, 
          epochs=50, 
          validation_data=(X_test, y_test))

Epoch 1/50
1331/1331 [==============================] - 5s 4ms/step - loss: 112484.6406 - mse: 112484.6406 - val_loss: 1260.7158 - val_mse: 1260.7158
Epoch 2/50
1331/1331 [==============================] - 5s 4ms/step - loss: 7902.9468 - mse: 7902.9468 - val_loss: 15.1018 - val_mse: 15.1018
Epoch 3/50
1331/1331 [==============================] - 5s 4ms/step - loss: 9.0184 - mse: 9.0184 - val_loss: 4.2481 - val_mse: 4.2481
Epoch 4/50
1331/1331 [==============================] - 6s 4ms/step - loss: 1.9767 - mse: 1.9767 - val_loss: 0.7793 - val_mse: 0.7793
Epoch 5/50
1331/1331 [==============================] - 6s 4ms/step - loss: 0.5743 - mse: 0.5743 - val_loss: 0.5165 - val_mse: 0.5165
Epoch 6/50
1331/1331 [==============================] - 6s 4ms/step - loss: 0.5143 - mse: 0.5143 - val_loss: 0.5135 - val_mse: 0.5135
Epoch 7/50
1331/1331 [==============================] - 5s 4ms/step - loss: 0.5142 - mse: 0.5142 - val_loss: 0.5139 - val_mse: 0.5139
Epoch 8/50
1331/1331 [================

In [33]:
np.expm1(0.5141)



0.6721329047662037

In [32]:
dir(np)

['ALLOW_THREADS',
 'AxisError',
 'BUFSIZE',
 'CLIP',
 'ComplexWarning',
 'DataSource',
 'ERR_CALL',
 'ERR_DEFAULT',
 'ERR_IGNORE',
 'ERR_LOG',
 'ERR_PRINT',
 'ERR_RAISE',
 'ERR_WARN',
 'FLOATING_POINT_SUPPORT',
 'FPE_DIVIDEBYZERO',
 'FPE_INVALID',
 'FPE_OVERFLOW',
 'FPE_UNDERFLOW',
 'False_',
 'Inf',
 'Infinity',
 'MAXDIMS',
 'MAY_SHARE_BOUNDS',
 'MAY_SHARE_EXACT',
 'MachAr',
 'ModuleDeprecationWarning',
 'NAN',
 'NINF',
 'NZERO',
 'NaN',
 'PINF',
 'PZERO',
 'RAISE',
 'RankWarning',
 'SHIFT_DIVIDEBYZERO',
 'SHIFT_INVALID',
 'SHIFT_OVERFLOW',
 'SHIFT_UNDERFLOW',
 'ScalarType',
 'Tester',
 'TooHardError',
 'True_',
 'UFUNC_BUFSIZE_DEFAULT',
 'UFUNC_PYVALS_NAME',
 'VisibleDeprecationWarning',
 'WRAP',
 '_NoValue',
 '_UFUNC_API',
 '__NUMPY_SETUP__',
 '__all__',
 '__builtins__',
 '__cached__',
 '__config__',
 '__doc__',
 '__file__',
 '__git_revision__',
 '__loader__',
 '__name__',
 '__package__',
 '__path__',
 '__spec__',
 '__version__',
 '_add_newdoc_ufunc',
 '_distributor_init',
 '_global

In [20]:
import pickle 
  
# Save the trained model as a pickle string. 
saved_model = pickle.dumps(model) 
  
# Load the pickled model 
model_from_pickle = pickle.loads(saved_model) 
  
# Use the loaded pickled model to make predictions 
model_from_pickle.predict(X_test) 

TypeError: ignored

In [37]:
import joblib 
  
# Save the model as a pickle in a file 
joblib.dump(model, 'kristine_model_0.pkl') 
  
# Load the model from the file 
model_from_joblib = joblib.load('kristine_model_0.pkl')  
  
# Use the loaded model to make predictions 
model_from_joblib.predict(X_test) 

TypeError: ignored

In [41]:
kristine_models = model.save('kristine_model_0')

INFO:tensorflow:Assets written to: kristine_model_0/assets


In [43]:
test_model = tf.keras.models.load_model('kristine_model_0')

In [44]:
test_model

In [46]:
test_model.predict(X_test)

array([[4.765928],
       [4.765928],
       [4.765928],
       ...,
       [4.765928],
       [4.765928],
       [4.765928]], dtype=float32)

In [48]:
X_test.head(10)

room_type  accommodates  bathrooms  ...  beds   TV  zipcode
9378         1.0           5.0        1.0  ...   3.0  2.0      0.0
53123        2.0           1.0        1.0  ...   2.0  1.0      0.0
32033        2.0           2.0        1.0  ...   1.0  1.0      0.0
40652        3.0           2.0        1.0  ...   1.0  1.0      0.0
32607        1.0           2.0        1.0  ...   1.0  2.0      0.0
32666        1.0           3.0        1.0  ...   1.0  1.0      0.0
49949        1.0           3.0        1.0  ...   1.0  2.0      0.0
14158        1.0           2.0        1.0  ...   1.0  2.0      0.0
22059        3.0           2.0        1.0  ...   1.0  1.0      0.0
22776        3.0           1.0        1.0  ...   1.0  2.0      0.0

[10 rows x 10 columns]